# Exercise 04: Bayesian Inverse Problems

In this notebook, we explore uncertainty quantification for inverse problems through Bayesian inference.

Essentially, inverse problems aim to infer the *parameters* of an unknown object given a physical model and *observed data*. Within the realm of uncertainty quantification, the focus is on the uncertainty associated with these inferred parameters.

In the Bayesian framework, uncertainty is quantified by a probability distribution over the parameter space. To estimate this uncertainty, Bayesian inference utilizes Bayes' theorem to define the *posterior distribution* of the parameters by combining prior knowledge and observed data.

This notebook aims to demonstrate the use of CUQIpy to combine the all the components required for Bayesian inference in a way that closely matches the mathematical formalism.

The online documentation for CUQIpy can be found at [https://cuqi-dtu.github.io/CUQIpy/](https://cuqi-dtu.github.io/CUQIpy/). Throughout the notebook, we will link to the relevant sections of the documentation for further details.

## Learning objectives
Going through the notebook, you will see how to quantify the uncertainty for an example inverse problem using the CUQIpy library by learning how to:

* Define distributions for each of the relevant parameters.
* Define a Bayesian inverse problem by combining distributions into a joint distribution.
* Construct a posterior distribution by conditioning the joint distribution on observed data.
* Sample a posterior distribution with specific choice of sampler.
* Analyze the samples from the posterior distribution.
* Compute point estimates of posterior, e.g., MAP or ML.
* Describe how the high-level "BayesianProblem" combines the above steps into a convenient non-expert interface.


## Table of contents
1. [Defining the posterior distribution](#posterior)
2. [Sampling the posterior](#sampling)
3. [High level interface (BayesianProblem)](#BayesianProblem)
4. [Computing point estimates of the posterior](#pointestimates) ★

The section marked with ★ is optional and can be skipped if you are short on time.

## References
[1] Gelman, Andrew, et al. "Bayesian workflow." arXiv preprint arXiv:2011.01808 (2020) https://arxiv.org/abs/2011.01808.

[2] Riis, Nicolai AB, et al. "CUQIpy--Part I: computational uncertainty quantification for inverse problems in Python." arXiv preprint arXiv:2305.16949 (2023) https://arxiv.org/abs/2305.16949.


## Load modules
We start of by importing the Python packages we need (including CUQIpy).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import cuqi
from cuqi.testproblem import Deconvolution1D, Deconvolution2D
from cuqi.distribution import JointDistribution, Gaussian, CMRF, LMRF, GMRF
from cuqi.sampler import Linear_RTO, pCN, CWMH, ULA, MALA, NUTS
from cuqi.problem import BayesianProblem


# 1. Defining the posterior distribution <a class="anchor" id="posterior"></a>

Solving a Bayesian inverse problem amounts to characterizing the posterior distribution. The posterior describes the probability distribution of the parameters we are interested in by combining our prior knowledge of the parameters with the data we have observed. In its most general form, the posterior is given by Bayes' theorem:

\begin{align*}
p(\boldsymbol{\theta} \mid \mathbf{y}) = \frac{p(\mathbf{y} \mid \boldsymbol{\theta})p(\boldsymbol{\theta})}{p(\mathbf{y})} \propto p(\mathbf{y} \mid \boldsymbol{\theta})p(\boldsymbol{\theta}),
\end{align*}

where $\boldsymbol{\theta}$ is the parameter vector of *all* the parameters we are interested in inferring and $\mathbf{y}$ is the observable data. 

The probability density function $p(\boldsymbol{\theta})$ is the prior distribution of the parameters and $p(\mathbf{y} \mid \boldsymbol{\theta})$ is known as the likelihood function, often denoted as $L(\boldsymbol{\theta} \mid \mathbf{y})$ to emphasize its a function of $\boldsymbol{\theta}$ in this context. The denominator $p(\mathbf{y})$ is the *evidence* and is a normalization constant (that we typically ignore because it does not affect the MCMC sampling) that ensures that the posterior integrates to 1.


### Note on Bayesian modelling with CUQIpy

CUQIpy uses a general approach to Bayesian modeling that not only aims to define the posterior distribution, but also to define the joint distribution of all the parameters. This more general approach is useful because it allows one to carry out more tasks related to uncertainty quantification of inverse problems such as prior-predictive checks, model checking, posterior-predictive checks and more. For more details on some of these topics see the overview in [1].

In this notebook, we will only focus on defining the posterior distribution for an inverse problem with the purpose of quantifying the uncertainty of the inferred parameters, which is the main functionality of CUQIpy.

In short, the posterior is defined by *conditioning* the joint distribution of all the random variables on the observed data using Bayes' theorem and can be summarized in the following steps:

1. Define deterministic forward models and prepare observed data.
2. Define distributions for each of the parameters and data.
4. Combine the distributions into a joint distribution.
5. Condition the joint distribution on the observed data to obtain the posterior distribution.

We now go through these steps in more detail.

## 1.1 Deterministic forward model and observed data
Consider a Bayesian inverse problem
$$
\mathbf{y}=\mathbf{A}\mathbf{x} + \mathbf{e},
$$

where $\mathbf{A}: \mathbb{R}^n \to \mathbb{R}^m$ is the (deterministic) forward model of the inverse problem and $\mathbf{y}$ and $\mathbf{x}$ are random variables representing the observed data and parameter of interest respectively. Here $\mathbf{e}$ is a random variable representing additive noise in the data.

For this example let us consider the `Deconvolution1D` testproblem and extract a CUQIpy forward model and some synthetic data denoted $\mathbf{y}_\mathrm{data}$ (a realization of $\mathbf{y}$). 

Note that this is a linear inverse problem, but the same approach can be used for nonlinear inverse problems.

In [ ]:
# Load forward model, data and problem information
A, y_data, probInfo = Deconvolution1D.get_components(phantom="sinc")

# For convenience, we define the dimension of the domain of A
n = A.domain_dim

# For convenience, we extract the exact solution as x_exact
x_exact = probInfo.exactSolution


Before going further let us briefly visualize the data and compare with the exact solution to the problem.

Here we should expect to see that the data is a convolved version of the exact solution with some added noise. We can also inspect the `probInfo` variable to get further information about the problem.

In [ ]:
# Plot the data
plt.subplot(121); x_exact.plot(); plt.title('Exact Solution')
plt.subplot(122); y_data.plot(); plt.title('Data')

# Print information about the problem
print(probInfo)


## 1.2 Distributions for parameters and data

The goal now is to define distributions for each the random variables that represent the parameters and data in the above-mentioned Bayesian inverse problem. One way to think about this is that we are trying to define a statistical model for the data generating process (see reference [1]), which we call a *Bayesian Problem* in CUQIpy to distinguish it from forward models.

For the unknown $\mathbf{x}$, we use a-priori knowledge to define its distribution. In this case, the sinc phantom in the Deconvolution test problem can be represented fairly well by a distribution that generates smooth, but spatially correlated realizations.

One such distribution is the Gaussian Markov Random Field (GMRF) distribution. This distribution assumes a Gaussian distribution on the differences between neighboring elements of $\mathbf{x}$, i.e. in 1D:

\begin{align*}
x_i - x_{i-1} \sim \mathcal{N}(0, d^{-1}), \quad i=1, \ldots, n,
\end{align*}

where we purposely leave out the details on the boundary conditions for this notebook.

To simplify the notation, we denote by *GMRF* the distribution that induces this property on a vector $\mathbf{x}$ defined by its mean and precision $d$. That is, the above can be written as

\begin{align*}
\mathbf{x} &\sim \mathrm{GMRF}(\mathbf{0}, d),
\end{align*}


with some choice of the precision say $d=50$. For more details on GMRF see the first CUQIpy paper [2].

The GMRF distribution is implemented in CUQIpy as [GMRF class](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.GMRF.html#cuqi.distribution.GMRF) and can be used as follows:

In [ ]:
# Define prior precision
d = 50

# Define GMRF prior (zero boundary conditions are the default)
x = GMRF(np.zeros(n), d)


From the problem info string, we see that the noise is Gaussian with standard deviation $s_\mathbf{e}=0.01$, i.e.,

\begin{align*}
\mathbf{e} &\sim \mathcal{N}(\mathbf{0}, s_\mathbf{e}^2 \mathbf{I}).
\end{align*}

Instead of working directly with the noise variable $\mathbf{e}$, we can include the stochastic element of the noise in the random variable $\mathbf{y}$ representing the data.

Because the data depend on $\mathbf{x}$ we are really interested in defining the *data distribution* for $\mathbf{y} \mid \mathbf{x}$ and since the noise is the only stochastic element of $\mathbf{y}$ when $\mathbf{x}$ is fixed we have

\begin{align*}
\mathbf{y} \mid \mathbf{x} &\sim \mathcal{N}(\mathbf{A}\mathbf{x}, s_\mathbf{y}^2 \mathbf{I}),
\end{align*}

where $s_\mathbf{e}=s_\mathbf{y} = 0.01$.

Notice that this definition depends both on the forward model $\mathbf{A}$ and the random variable $\mathbf{x}$.

In CUQIpy, we can define the distribution for $\mathbf{y} \mid \mathbf{x}$ matching the mathematical expression using our previously defined variables `A` and `x` as follows:

In [ ]:
# Define noise standard deviation
s_y = 0.01

# Define data distribution
y = Gaussian(A@x, s_y**2)


#### ★ Try yourself (optional):  

Have a look at the distributions for $\mathbf{x}$ and $\mathbf{y}$ by calling `print` on them in the code-cell below. 
- How are the distributions different?
- Is it clear that the distribution for $\mathbf{y}$ is a conditional distribution?
- Can you generate and plot a realization (sample) of $\mathbf{x}$? Does the realization show spatial correlation?
- How about generating a sample from $\mathbf{y}\mid\mathbf{x}$? What does a sample represent in this case?

**Hint:** to condition `y` on a value e.g. `x_exact` the syntax is `y(x=x_exact)`

In [ ]:
# Your code here




### Note on notation

It is common (for convenience in terms of notation) not to explicitly write the dependance of each random variable when specifying a complete Bayesian problem. For example, for the case above one would often write
\begin{align*}
\mathbf{x} &\sim \mathrm{GMRF}(\mathbf{0}, d)\\
\mathbf{y} &\sim \mathcal{N}(\mathbf{A}\mathbf{x}, s_\mathbf{y}^2 I),
\end{align*}

where the dependance of $\mathbf{y}$ on $\mathbf{x}$ is implicit.

This compact notation completely specifies the Bayesian problem for the so-called *data generating process*, making clear all the assumptions about the data and parameters.

In CUQIpy - when all deterministic parameters and forward models are defined - the Bayesian problem is written in code using almost exactly the same syntax as the mathematical notation above.

In [ ]:
# Bayesian problem (repeated in case previous cells were modified)
x = GMRF(np.zeros(n), 50)
y = Gaussian(A@x, s_y**2)


## 1.3 Joint distribution

In the Bayesian framework we are interested in the joint distribution over $\mathbf{x}$ and $\mathbf{y}$. The joint distribution is defined as the product of the individual probability density functions. In our simple case, the joint distribution can be described in density form as

$$
p(\mathbf{y},\mathbf{x}) = p(\mathbf{y} \mid \mathbf{x})p(\mathbf{x}).
$$

In CUQIpy, we can define the joint distribution simply by passing each of the distributions as arguments to the `JointDistribution` class.

In [ ]:
# Define joint distribution p(y,x)
joint = JointDistribution(y, x)


Calling `print` on the joint distribution gives a nice overview matching the mathematical description above.

In [ ]:
print(joint)


For the purpose of this notebook we are not going to dive further into the details of the joint distribution, and simply use it to define the posterior.

## 1.4 Posterior

The final step in defining the posterior distribution is to condition the joint distribution $p(\mathbf{y},\mathbf{x})$ on the observed data, say $\mathbf{y}_\mathrm{data}$. This is done by using Bayes' theorem, which in our simple case can easily be derived as

\begin{align*}
p(\mathbf{x} \mid \mathbf{y}) \propto L(\mathbf{x} \mid \mathbf{y})p(\mathbf{x}),
\end{align*}

where we use the notation $L(\mathbf{x} \mid \mathbf{y}) := p(\mathbf{y} \mid \mathbf{x})$ for the likelihood function to emphasize that, in the context of the posterior where $\mathbf{y}$ is fixed, it is a function of $\mathbf{x}$ and not on $\mathbf{y}$.

CUQIpy can automatically derive the posterior distribution for any joint distribution by passing the observed data as an argument to the "call" (condition) method of the joint distribution. This is done as follows:

In [ ]:
posterior = joint(y=y_data) # Condition p(x,y) on y=y_data. Applies Bayes' rule automatically


We can now inspect the posterior distribution by calling `print` on it. Notice that the posterior equation matches the mathematical expression above.

In [ ]:
print(posterior)


#### ★ Try yourself (optional):  
The posterior is essentially just another CUQIpy distribution. Have a look at the [Posterior class](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.Posterior.html) in the online documentation to see what attributes and methods are available.

Try evaluating the posterior log probability density function (logpdf) at some point say $\mathbf{1}$ and $2\cdot\mathbf{1}$, where $\mathbf{1}$ is a ones-vector.

Try evaluating the posterior pdf at the same points. Can you explain why the pdf gives the same result for both points?

In [ ]:
# Your code here




# 2. Sampling the posterior <a class="anchor" id="sampling"></a>

Now that we have defined the posterior distribution for our parameter of interest $\mathbf{x}$ given $\mathbf{y}_\mathrm{data}$, we can characterize the parameter and its uncertainty by samples from the posterior distribution. However, in general the posterior is not a simple distribution that we can easily sample from. Instead, we need to rely on Markov Chain Monte Carlo (MCMC) methods to sample from the posterior.

In CUQIpy, a number of MCMC samplers are provided in the sampler module that can be used to sample probability distributions. All samplers have the same signature, namely `Sampler(target, ...)`, where `target` is the target CUQIpy distribution and `...` indicates any (optional) arguments.

In the case of the posterior above, which is defined from a linear model and Gaussian likelihood and prior, the Linear Randomize-then-Optimize [(Linear_RTO)](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.sampler/cuqi.sampler.Linear_RTO.html#cuqi.sampler.Linear_RTO) sampler is a good choice to efficiently generate samples. Like any of the other samplers, we set up the sampler by simply providing the target distribution.

In [ ]:
sampler = Linear_RTO(posterior)


After the sampler is defined we can compute samples via the `sample` method.

In [ ]:
samples = sampler.sample(500)


Similar to directly sampling distributions in CUQIpy, the returned object is a `cuqi.samples.Samples` object.

This object has a number of methods available. In this case, we are interested in evaluating if the sampling went well. To do this we can have a look at the chain history for 2 different parameters.

In [ ]:
samples.plot_chain([30, 45]);


In both cases the chains look very good with no discernible difference between the start and end of the chain. This is a good indication that the chain has converged and there is little need for removing samples that are part of a "burn-in" period. In practice, the samples should be inspected with more rigor to ensure that the MCMC chain has converged, but this is outside the scope of this notebook.

The good sampling is in large part due to the Linear_RTO sampler, which is built specifically for the type of problem of this example. For the sake of presentation let us remove the first 100 samples using the `burnthin` method (see [samples.burnthin](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.samples/cuqi.samples.Samples.burnthin.html#cuqi.samples.Samples.burnthin)) and store the "burnthinned" samples in a new variable.

In [ ]:
samples_final = samples.burnthin(Nb=100)


Finally, we can plot a credibility interval of the samples and compare to the exact solution (from `probInfo`)

In [ ]:
samples_final.plot_ci(95, exact=probInfo.exactSolution)


### Trying out other samples

The Linear_RTO sampler can only sample Gaussian posteriors that also have an underlying linear model.

It is possible to try out other CUQIpy samplers (which also work for a broader range of problems). For example:

* **[pCN](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.sampler/cuqi.sampler.pCN.html#cuqi.sampler.pCN)** - preconditioned Crank-Nicolson sampler.
* **[CWMH](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.sampler/cuqi.sampler.CWMH.html)** - Component-wise Metropolis-Hastings sampler.
* **[ULA](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.sampler/cuqi.sampler.ULA.html)** - Unadjusted Langevin Algorithm.
* **[MALA](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.sampler/cuqi.sampler.MALA.html)** - Metropolis Adjusted Langevin Algorithm.
* **[NUTS](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.sampler/cuqi.sampler.NUTS.html)** - No U-Turn Sampler: A variant of the Hamiltonian Monte Carlo sampler well-established in literature.

Note in particular that ULA, MALA and NUTS all require the gradient of the logpdf. This is handled automatically in CUQIpy for linear models.

#### ★ Try yourself (optional):  

Try sampling the posterior above using one of the suggested samplers (click the links to look at the online documentation for the sampler to get more info on it).

Compare results (chain, credibility interval etc.) to the results from Linear_RTO.

All the suggested samplers (except NUTS) will likely require > 5000 samples to give reasonable results, and for some playing with step sizes (scale) is needed. This is because they are not as efficient as Linear_RTO or NUTS. For some samplers, the method [sample_adapt](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.sampler/cuqi.sampler.NUTS.sample_adapt.html#cuqi.sampler.NUTS.sample_adapt) will auto-scale the step size according to some criteria, e.g. reach approximately optimal acceptance rate and a burn-in should be added to specify how many samples to use for the adaptation.



In [ ]:
# Your code here





# 3. High-level interface (BayesianProblem) <a class="anchor" id="BayesianProblem"></a>

We now make the connection to the [`BayesianProblem`](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.problem/cuqi.problem.BayesianProblem.html#cuqi.problem.BayesianProblem) class in CUQIpy that provides a non-expert interface for Bayesian inference.

The Bayesian Problem class tries to conveniently wrap most of the steps we have seen in this notebook into a single object.

What is needed is to define the deterministic forward model and distributions for each of the parameters. The rest is handled by the Bayesian Problem class as exemplified below.

Note that the `BayesianProblem` class automatically defines the posterior distribution and chooses a sampler based on the problem type. 
*The automatic sampler selection is still work-in-progress and one of the goals of CUQI project is to determine best choices of samplers for a variety of inverse problems.*

In [ ]:
# Deterministic forward model
A, y_data, probInfo = Deconvolution1D.get_components(phantom="sinc")

# Distributions for each parameter
x = GMRF(np.zeros(A.domain_dim), 50)
y = Gaussian(A@x, 0.01**2)

# Define Bayesian problem, set data and do UQ!
BP = BayesianProblem(y, x).set_data(y=y_data)
samples = BP.UQ(exact=probInfo.exactSolution)


One can also use the [`sample_posterior`](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.problem/cuqi.problem.BayesianProblem.sample_posterior.html#cuqi.problem.BayesianProblem.sample_posterior) method to get samples of the posterior without the UQ plots.

Similar to distributions and samplers, the sampling using `BayesianProblem` returns a `cuqi.samples.Samples` object so we can e.g. plot the credibility interval or chains easily.

In [ ]:
samples.plot_chain([30, 45]);


####  Try yourself (optional):  

- Try another prior such as [LMRF](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.LMRF.html#cuqi.distribution.LMRF) or [CMRF](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.distribution/cuqi.distribution.CMRF.html#cuqi.distribution.CMRF) for the 1D case (look up appropriate arguments in the documentation) using [`BayesianProblem`](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.problem/cuqi.problem.BayesianProblem.html#cuqi.problem.BayesianProblem).
- Try switching the testproblem from Deconvolution1D to [Deconvolution2D](https://cuqi-dtu.github.io/CUQIpy/api/_autosummary/cuqi.testproblem/cuqi.testproblem.Deconvolution2D.html#cuqi.testproblem.Deconvolution2D) (look up appropriate arguments in the documentation).
- You may also try defining your own Bayesian inverse problem using this interface. ★

In [ ]:
# Your code here
# You can modify this code or write your own from scratch

# 1. Forward model and data
A, y_data, probInfo = Deconvolution1D.get_components(phantom="sinc")

# 2. Distributions
x = GMRF(np.zeros(A.domain_dim), 50) # Try e.g. LMRF or CMRF (also update parameters!)
y = Gaussian(A@x, 0.01**2)

# 3. Bayesian problem
BP = BayesianProblem(y, x).set_data(y=y_data)

# 4. Sample posterior
samples = BP.sample_posterior(500)

# 5. Analyze posterior
samples.plot_ci(exact=probInfo.exactSolution)


You may have noticed that finding suitable parameters for the prior could be a challenge. To see how to automatically find suitable parameters for the prior, see the [Gibbs notebook](Exercise06_Gibbs.ipynb).

# 4. Computing point estimates of the posterior ★ <a class="anchor" id="pointestimates"></a>

In addition to sampling the posterior, we can also compute point estimates of the posterior. A common point estimate to consider is the Maximum A Posteriori (MAP) estimate, which is the value of the Bayesian parameter that maximizes the posterior density. That is,

\begin{align*}
\mathbf{x}_\mathrm{MAP} = \arg\max_\mathbf{x} p(\mathbf{x} \mid \mathbf{y}_\mathrm{data}).
\end{align*}

The easiest way to compute the MAP estimate is to use the `MAP` method of the `BayesianProblem` class as follows:

In [ ]:
# We redefine in case something was changed

# Deterministic forward model
A, y_data, probInfo = Deconvolution1D.get_components(phantom="sinc")

# Distributions for each parameter
x = GMRF(np.zeros(A.domain_dim), 50)
y = Gaussian(mean=A@x, cov=0.01**2)

# Define Bayesian problem
BP = BayesianProblem(y, x).set_data(y=y_data)


In [ ]:
x_map = BP.MAP() # Maximum a posteriori estimate


The automatic solver selection is also still work-in-progress.

After we have computed the MAP, we can then estimate to the exact solution (from `probInfo`)

In [ ]:
x_map.plot()
plt.title('MAP estimate')
plt.show()

probInfo.exactSolution.plot()
plt.title('Exact solution')
plt.show()


#### ★ Try yourself (optional):  

- Try switching to the Deconvolution2D testproblem. You may have to play with the prior standard deviation to get a good MAP estimate.
- Try switching the prior to a CMRF distribution for the 1D case. Does the MAP estimate change?